In [5]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import Siren, NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold 
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic

from phimal_utilities.data import Dataset
from phimal_utilities.data.burgers import BurgersDelta
from experiment_code import train, train_bayes_MSE_optim, train_bayes_full_optim

if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.2, random=True, normalize=False)
X, y = X.to(device), y.to(device)
        


In [7]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500000, delta=0.0) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer


In [8]:
train_bayes_full_optim(model, X, y, optimizer, sparsity_scheduler, log_dir='burgers/correct/', max_iterations=50000, delta=0.0)

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
      31625     63.25%             717s   -9.38e+03   1.43e-03   1.90e-10   3.29e+00 

KeyboardInterrupt: 

In [56]:
data_train = X
target_train = y

In [59]:
prediction, time_derivs, thetas = model(data_train)
t = time_derivs[0]
Theta = thetas[0]

tau_MSE = torch.exp(model.s[:, 0])
alpha = torch.exp(model.s[:, 1])#torch.exp(model.s[:, 1]) #1 / MSE[0].data
beta = torch.exp(model.s[:, 2])#torch.exp(model.s[:, 2]) #torch.tensor(1e-5).to(Theta.device)

M = thetas[0].shape[1]

Sn = torch.inverse(torch.eye(M).to(Theta.device) * alpha + beta * Theta.T @ Theta)
mn = beta * Sn @ Theta.T @ t
tau_reg = torch.trace(torch.log(1/beta + Theta @ Sn @ Theta.T))

MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
Reg = torch.mean((t - Theta @ mn)**2, dim=0)


loss = torch.sum(tau_MSE * MSE - torch.log(tau_MSE) + tau_reg * Reg - torch.log(tau_reg)) 


In [62]:
tau_MSE

tensor([810.6118], device='cuda:0', grad_fn=<ExpBackward>)

In [63]:
beta

tensor([0.1065], device='cuda:0', grad_fn=<ExpBackward>)

In [64]:
tau_reg

tensor(2250.2727, device='cuda:0', grad_fn=<TraceBackward>)

In [38]:
tau_reg

tensor(1453.3552, device='cuda:0', grad_fn=<TraceBackward>)

In [26]:
model.estimator_coeffs()

[array([[ 0.        ],
        [ 0.        ],
        [ 0.59891641],
        [ 0.        ],
        [ 0.        ],
        [-0.97883278],
        [ 0.        ],
        [ 1.11279523],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-1.02177572]])]

In [84]:
tau_reg * Reg - torch.log(tau_reg)

tensor([0.6570], device='cuda:0', grad_fn=<SubBackward0>)

In [74]:
mn

tensor([[ 0.0044],
        [-0.2736],
        [ 0.1210],
        [ 0.0039],
        [-0.0023],
        [ 0.6976],
        [-0.3757],
        [-0.0196],
        [-0.1111],
        [-2.2120],
        [ 0.4564],
        [ 0.0320]], device='cuda:0', grad_fn=<MmBackward>)

In [85]:
tau_MSE * MSE - torch.log(tau_MSE)

tensor([-5.6653], device='cuda:0', grad_fn=<SubBackward0>)

In [68]:
from sklearn.linear_model import BayesianRidge

In [77]:
reg = BayesianRidge(fit_intercept=False, compute_score=True)

In [78]:
reg.fit(Theta.cpu().detach().numpy(), t.cpu().detach().numpy())

BayesianRidge(compute_score=True, fit_intercept=False)

In [86]:
reg.coef_

array([ 0.00444178, -0.28377185,  0.12149715,  0.00404318, -0.00377249,
        0.75102072, -0.38060624, -0.02000639, -0.10867087, -2.26315693,
        0.46189379,  0.03232521])

In [87]:
reg.scores_

array([ 403.40620505, 1313.42762996, 1313.74925292, 1313.75015703,
       1313.75015925, 1313.75017231])

In [88]:
reg.alpha_

265.7060810458401

In [89]:
reg.lambda_

1.9426210685039595

So somethings wrong with my algo.